In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import copy
import scipy.stats as stats
from scipy.stats import gumbel_r

In [2]:
np.random.seed(1)

In [3]:
random.seed(1)

In [4]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [5]:
def collection_distribution_hev(n,collection,price):
    # generate mean 
    ''' mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1) '''
    # generate deterministic utilities for products
    
    ''' rho = -0.5 # prices and utilities are negatively correlated
    price_mean = np.mean(price[1:]) 
    price_std = np.std(price[1:])
    
    z = np.zeros(len(price[1:]))
    for i in range(len(z)):
        z[i] = (price[1:][i] - price_mean)/price_std
        
    mu_1n = np.zeros(len(z))
    normal_rvs = np.random.randn(len(mu_1n))
    for i in range(len(mu_1n)):
        mu_1n[i] = rho*z[i] + np.sqrt(1-rho**2)*normal_rvs[i] 
    ## deterministic utility of the outside option is strictly greater than the utilities of the products
    mu_0 = np.max(mu_1n) + 1
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = 10
    scale_1n = np.random.uniform(0.04,1,n-1)
    scales = np.hstack((scale_0,scale_1n)) '''
    
    mu = np.random.uniform(-2,2,n)
    scales = np.random.uniform(0.04,2,n)
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])
    # generate covariance matrix with positive correlation
    # neg_cov = generate_negatively_correlated_covariance_matrix(n)
    # if is_positive_semidefinite(neg_cov): 
    #     sample_size = 10000
    #     utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    # else:
    #     print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [6]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [100,100,100,100,100]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [7]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [8]:
all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [9]:
def filter_probability(whole_choice_collection,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
 
        
    return choice_collection

In [10]:
## generate 50 random full instances 
pred_full_instance_indep = []

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    indep_curr_whole_instance = collection_distribution_hev(n,jd_collection,price)

    pred_full_instance_indep.append(indep_curr_whole_instance)


generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [11]:
# full instance generation for indep gaussian 
all_full_instances_indep = []
 
for i in range(len(pred_offer_times_list)):
    
    full_instances_indep = []

    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection_indep = filter_probability(pred_full_instance_indep[j],all_full_assortment_index[i])
        full_instances_indep.append(curr_choice_collection_indep)
     
    all_full_instances_indep.append(full_instances_indep)

check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [12]:
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value

In [13]:
def mdm_feasibility(data,ranking,equal):
      
    ub = 100

    model = gp.Model('mdm')
    #model.setParam(gp.GRB.Param.TimeLimit, 60)
    model.setParam('OutputFlag', 0)
    eps = model.addVar(name='eps')
    lam = model.addVars(data.shape[1],name = 'lam')
    
    model.addConstr(eps<=ub ) # just give an upper bound (+ve) for eps, o.w., the problem is unbounded
    
    #for i in range(ranking.shape[0]): # from each product 
    for i in range(len(ranking)): # from each product 
        for j in range(len(ranking[i])-1): # the ranking of assortment in product i 
            model.addConstr(lam [ranking[i][j+1]] - lam[ranking[i][j]]-  eps >=0)
    
    for i in range(len(equal)):
        if len(equal[i])>0:
            for j in range(len(equal[i][0])-1):
                model.addConstr(lam[equal[i][0][j]]-lam[equal[i][0][j+1]]==0)
    
    model.setObjective(eps,GRB.MAXIMIZE)
    model.optimize()
    
     # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        #print('Optimal solution found!')
        eps_value = eps.x
        if eps_value>0:
            return [1,model.Runtime]
        else:
            #model.write("mdminfeas_model.lp")
        
            return [0,model.Runtime]
    else:
        print('MDM feasibility formulation error')
    

In [14]:
def mdm_limit(data, collection,offer_times):
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.01

    model = gp.Model()
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")
    
    # Define auxiliary binary variables for absolute value terms
    abs_vars = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="abs")
    ''' abs_vars = {}
    for i in range(n):
        for j in range(m):
            abs_vars[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"abs_{i}_{j}") '''

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{j}_{0}")
                #model.addConstr(abs_vars[j, i] == 0, name=f"abs_forcing_zero_{j}_{0}")
            else:
                model.addConstr(x[j, i] - data[j, i] <= abs_vars[j, i], name=f"abs_const1_{j}_{i}")
                model.addConstr(-x[j, i] + data[j, i] <= abs_vars[j, i], name=f"abs_const2_{j}_{i}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")

    # Add linear constraints to represent absolute value terms
    ''' for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= abs_vars[i, j], name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= abs_vars[i, j], name=f"abs_const2_{i}_{j}") '''

    # Define the modified objective function using auxiliary variables
    obj = (sum(offer_times[j]*data[i, j] * abs_vars[i, j] for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))

    model.setObjective(obj, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()
    
    x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [lam[i].x for i in range(m)]
    lam_values = np.array(lam_values)

    if model.Status == GRB.OPTIMAL:
        print("Gurobi optimization status:", model.status)
        return [model.objVal, model.Runtime,lam_values,x_values]
    else:
        print("Gurobi optimization status:", model.status)
        model.write("mdmlimit_model.lp") 
        #return [model.objVal, model.Runtime,lam_values,x_values]

In [15]:
## record representability of all instances
all_mdm_rep_indep = []
all_mdm_rep_time_indep = []

## record the limit loss of all instances
all_mdm_limit_loss_indep = []
all_mdm_limit_prob_indep = []
all_mdm_limit_runtime_indep = []

all_mdm_limit_lam_indep = []


for i in range(len(pred_offer_times_list)):

    ## define container for mdm representability check
    mdm_rep_collection_indep =[]
    mdm_rep_runtime_collection_indep =[]
    
    ## define container for mdm limit 
    mdm_limit_loss_collection_indep =[]
    mdm_limit_runtime_collection_indep =[]
    mdm_limit_probability_collection_indep =[]
    mdm_limit_lam_collection_indep = []
       
    for j in range(pred_instance_size[i]):
        print(f'testing instance with offertimes {pred_offer_times_list[i]} : {j}th ')
        ####### mdm representability check ##########
        
        row_data = row_assortment_data(all_full_instances_indep[i][j],all_full_collections[i])
        ## find the decreasing ranking and equal sequence of the assortment in the data
        rank, equal = find_indexes_nonnegative_descending(row_data)
        ## representability check of MDM
        curr_rep_result_indep = mdm_feasibility(all_full_instances_indep[i][j],rank,equal)
      
        # return [1/0,model.Runtime] # 1: representable 0: non-representable
        mdm_rep_collection_indep.append(curr_rep_result_indep[0])
        mdm_rep_runtime_collection_indep.append(curr_rep_result_indep[1])
        
        # check if representable
        if curr_rep_result_indep[0] > 0:
            mdm_limit_loss_collection_indep.append(0)
            mdm_limit_runtime_collection_indep.append(curr_rep_result_indep[1])
            mdm_limit_probability_collection_indep.append(all_full_instances_indep[i][j])
        else:
            # not representable # limit and prediction
            #mdm_limit(data,collection,perm_collection,produts_notin_collection)
            curr_limit_result_indep = mdm_limit(all_full_instances_indep[i][j],all_full_collections[i],all_full_offertimes[i])
            # return [model.objVal, model.Runtime,lam_values,x_values]
            curr_limit_prob = curr_limit_result_indep[-1]
            mdm_limit_probability_collection_indep.append(curr_limit_prob)
            mdm_limit_loss_collection_indep.append(curr_limit_result_indep[0])
            mdm_limit_runtime_collection_indep.append(curr_limit_result_indep[1])
            mdm_limit_lam_collection_indep.append(curr_limit_result_indep[2])
        print('the limit loss is', mdm_limit_loss_collection_indep[-1])
        print('the runtime is', mdm_limit_runtime_collection_indep[-1])   
    
    ## for each train instance [i][j] representability result is a number
    all_mdm_rep_indep.append(mdm_rep_collection_indep)
    all_mdm_rep_time_indep.append(mdm_rep_runtime_collection_indep)
    
    ## for each train instance [i][j] limit loss result is a number
    all_mdm_limit_loss_indep.append(mdm_limit_loss_collection_indep)
    all_mdm_limit_runtime_indep.append(mdm_limit_runtime_collection_indep)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_mdm_limit_prob_indep.append(mdm_limit_probability_collection_indep)
    
    all_mdm_limit_lam_indep.append(mdm_limit_lam_collection_indep)

testing instance with offertimes 20 : 0th 
Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic


Gurobi optimization status: 2
the limit loss is 0.23803718000000054
the runtime is 0.24168014526367188
testing instance with offertimes 20 : 1th 
Gurobi optimization status: 2
the limit loss is 0.005697869999999929
the runtime is 0.19816303253173828
testing instance with offertimes 20 : 2th 
Gurobi optimization status: 2
the limit loss is 0.3826081400000011
the runtime is 0.2364640235900879
testing instance with offertimes 20 : 3th 
Gurobi optimization status: 2
the limit loss is 0.08229073999999954
the runtime is 0.26821112632751465
testing instance with offertimes 20 : 4th 
Gurobi optimization status: 2
the limit loss is 0.14058162000000518
the runtime is 0.47646403312683105
testing instance with offertimes 20 : 5th 
Gurobi optimization status: 2
the limit loss is 3.564149669999998
the runtime is 0.6326980590820312
testing instance with offertimes 20 : 6th 
Gurobi optimization status: 2
the limit loss is 1.9460403599999971
the runtime is 0.6816699504852295
testing instance with offer

In [16]:
## adding rep results for indep
for i in range(len(pred_offer_times_list)):
    df_mdm_rep = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_rep':all_mdm_rep_indep[i], 'mdm_rep_time':all_mdm_rep_time_indep[i]})
    df_mdm_rep.to_csv('indep/representability/'+str(pred_offer_times_list[i])+'.csv')

## adding limit results
for i in range(len(pred_offer_times_list)):
    df_mdm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_loss':all_mdm_limit_loss_indep[i], 'mdm_limit_time':all_mdm_limit_runtime_indep[i]})
    df_mdm_loss.to_csv('indep/limit/'+str(pred_offer_times_list[i])+'.csv')
    
    for j in range(pred_instance_size[i]):
            df_limit_prob = pd.DataFrame(all_mdm_limit_prob_indep[i][j])
            df_limit_prob.to_csv('indep/limit/prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [17]:
all_avg_loss_indep = []
for i in range(len(pred_offer_times_list)):
    avg_loss_collection_indep = []
    for j in range(pred_instance_size[i]):
        avg_loss_collection_indep.append(all_mdm_limit_loss_indep[i][j]/sum(all_full_offertimes[i]))
    all_avg_loss_indep.append(avg_loss_collection_indep)

avg_mdm_rep_indep = []
avg_mdm_rep_runtime_indep = []
avg_mdm_rep_runtime__se_indep = []

avg_total_limit_loss_indep = []
avg_toal_limit_loss_se_indep = []
avg_mdm_limit_runtime_indep = []

#avg_loss = []
avg_loss_se_indep = []
avg_limit_runtime_se_indep = []

for i in range(len(pred_offer_times_list)):
    avg_mdm_rep_indep.append(np.mean(all_mdm_rep_indep[i]))
    avg_mdm_rep_runtime_indep.append(np.mean(all_mdm_rep_time_indep[i]))
    avg_mdm_rep_runtime__se_indep.append(np.std(all_mdm_rep_time_indep[i])/np.sqrt(len(all_mdm_rep_time_indep[i])))
    
    avg_total_limit_loss_indep.append(np.mean(all_mdm_limit_loss_indep[i]))
    avg_mdm_limit_runtime_indep.append(np.mean(all_mdm_limit_runtime_indep[i]))
    avg_toal_limit_loss_se_indep.append(np.std(all_mdm_limit_loss_indep[i])/np.sqrt(len(all_mdm_limit_loss_indep[i])))
    
    #avg_loss = np.mean(all_avg_loss[i])
    avg_loss_se_indep.append(np.std(all_avg_loss_indep[i])/np.sqrt(len(all_avg_loss_indep[i])))
    avg_limit_runtime_se_indep.append(np.std(all_mdm_limit_runtime_indep[i])/np.sqrt(len(all_mdm_limit_runtime_indep[i])))

avg_loss_indep = []
for i in range(len(avg_total_limit_loss_indep)):
    avg_loss_indep.append(avg_total_limit_loss_indep[i]/sum(all_full_offertimes[i]))

df_mdm_limit_indep = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'collection_size':full_collection_size,
                                   'indep_rep':avg_mdm_rep_indep,'indep_rep_runtime':avg_mdm_rep_runtime_indep,'indep_rep_runtime_se':avg_mdm_rep_runtime__se_indep,
                                   'indep_total_loss':avg_total_limit_loss_indep,'indep_total_loss_se':avg_toal_limit_loss_se_indep,
                                   'indep_avg_loss':avg_loss_indep,'indep_avg_loss_se':avg_loss_se_indep,
                                   'indep_limit_runtime':avg_mdm_limit_runtime_indep,'indep_limit_runtime_se':avg_limit_runtime_se_indep
                                   })
df_mdm_limit_indep.to_csv('indep/indep_limit_summary.csv')
df_mdm_limit_indep

,offertimes,collection_size,indep_rep,indep_rep_runtime,indep_rep_runtime_se,indep_total_loss,indep_total_loss_se,indep_avg_loss,indep_avg_loss_se,indep_limit_runtime,indep_limit_runtime_se
0,20,29,0.03,0.000337,0.000012,0.546409,0.074680,0.000071,0.000010,0.364822,0.026073
1,30,24,0.05,0.000382,0.000006,0.420489,0.065269,0.000056,0.000009,0.176604,0.015508
2,40,19,0.14,0.000335,0.000008,0.267636,0.058305,0.000036,0.000008,0.063629,0.004993
3,50,15,0.39,0.000203,0.000008,0.175779,0.043782,0.000024,0.000006,0.020611,0.001989
4,60,13,0.55,0.000157,0.000006,0.140255,0.041082,0.000020,0.000006,0.008363,0.001021
